# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

**Answer**  
This is a classification problem.  
The output of the problem is whether a student might need early intervention or not ('Yes' or 'No'), which is discreted.

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [30]:
# Import libraries
import numpy as np
import pandas as pd

In [31]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [32]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = (np.shape(student_data)[0])
n_features = (np.shape(student_data)[1] - 1)
n_passed = (len(student_data[(student_data['passed'] == 'yes')]))
n_failed = (len(student_data[(student_data['passed'] == 'no')]))
grad_rate = (float(len(student_data[(student_data['passed'] == 'yes')])) / float(n_students))  * 100.0
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 67.09%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [33]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [34]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [35]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=42)


print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

In [36]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.3f}".format(end - start)

# TODO: Choose a model, import it and instantiate an object
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
print clf  # you can inspect the learned model by printing it

Training DecisionTreeClassifier...
Done!
Training time (secs): 0.013
DecisionTreeClassifier(compute_importances=None, criterion='gini',
            max_depth=None, max_features=None, max_leaf_nodes=None,
            min_density=None, min_samples_leaf=1, min_samples_split=2,
            random_state=None, splitter='best')


In [37]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    return f1_score(target, y_pred, pos_label='yes')

train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.001
F1 score for training set: 1.0


In [38]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.65


In [39]:
# Train and predict using different training set sizes
from IPython.display import display

def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    start = time.time()
    train_classifier(clf, X_train, y_train)
    end = time.time()
    train_time = (end - start)
    train_f1_score = predict_labels(clf, X_train, y_train)
    test_f1_score = predict_labels(clf, X_test, y_test)
    print "F1 score for training set: {}".format(train_f1_score)
    print "F1 score for test set: {}".format(test_f1_score)
    return [len(X_train), train_f1_score, test_f1_score, train_time]

In [40]:
# Note: Keep the test set constant
# This is a function to run the help function three times with different training size #
def train_different_size(clf, X_train, y_train, X_test, y_test):
    result_statistic = []      # initialize a statistic array 
    for i in range(1, 4):
        trainingSize = (i*100)
        X_train_partial = X_train[:trainingSize, :]
        y_train_partial = y_train[:trainingSize]
        # Run the helper function for desired subsets of training data and push the result to the record array
        result_statistic.append(train_predict(clf, X_train_partial, y_train_partial, X_test, y_test))
    # convert the statistic record array to a panda dataframe 
    df_result_statistic = pd.DataFrame(result_statistic)
    
    # formatting the dataframe for visualizawtion 
    df_result_statistic.columns = ['training_size', 'train_f1_score', 'test_f1_score', 'training_time (secs)']
    df_result_statistic = df_result_statistic.set_index('training_size')
    return (df_result_statistic)

#Model I. Decision Tree Classifier
Decision Tree is generally used in data mining and decision analysis.  
For example, I run a shopping website. I have the personal information and the buying history of each user. I want to retrieve the features and the decision converage from the data that play important roles to decide the buying behaviour. 

* **Strength**
 * A decision tree can be visualized in graphics so that we can understand and interpret the data very well.
  
* **Weakness**
 * It may suffer from overfitting when we have too many features.

Besides knowing whether a student might need early intervention, I also want to know how and by what factor does the decision made. For example, the model might tell me Bob studies less than 15 minutes every day so he needs early intervention. It will be more valuable than just telling me that Bob needs early intervention. 

In [41]:
clf_decisionTree = DecisionTreeClassifier()
display(train_different_size(clf_decisionTree, X_train, y_train, X_test, y_test))

------------------------------------------
Training set size: 100
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.002
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 1.0
F1 score for test set: 0.620689655172
------------------------------------------
Training set size: 200
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.004
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 1.0
F1 score for test set: 0.727272727273
------------------------------------------
Training set size: 300
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.004
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
Predicting labels using Dec

,train_f1_score,test_f1_score,training_time (secs)
training_size,,,
100,1.0,0.620690,0.001920
200,1.0,0.727273,0.003848
300,1.0,0.656000,0.005312


#Model II. Naive Bayes
Naive Bayes is generally very useful on text classification problems such as email spam detection and classifying a piece of article from different categories (sports, entertainment, finance and so on)  

* **Strengths**
 * The feaures for text classification are usually very high in dimension. Naive Bayes performs well under this critera.
 * It is also good at dealing with noisy dataset. 

* **Weakness**
 * The features should be independent to each other to classify well.
 * In text classification problem. Naive Bayes does not care about the word order. For example: when we say 'coffee cup' and 'cup coffee', they are talking about two different things. 'Coffee cup' is the cup for coffee and 'Cup coffee' is a cup of coffee. However, Naive Bayes will give the same results for these two terms.
 
I want to use Naive Bayes on the data as the data contains 48 feature columns which is quite many I think.

In [42]:
from sklearn.naive_bayes import GaussianNB
clf_naiveBayes = GaussianNB()
display(train_different_size(clf_naiveBayes, X_train, y_train, X_test, y_test))

------------------------------------------
Training set size: 100
Training GaussianNB...
Done!
Training time (secs): 0.001
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.846715328467
F1 score for test set: 0.802919708029
------------------------------------------
Training set size: 200
Training GaussianNB...
Done!
Training time (secs): 0.000
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.840579710145
F1 score for test set: 0.724409448819
------------------------------------------
Training set size: 300
Training GaussianNB...
Done!
Training time (secs): 0.001
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.001
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
F1 score for training set: 

,train_f1_score,test_f1_score,training_time (secs)
training_size,,,
100,0.846715,0.802920,0.001021
200,0.840580,0.724409,0.000343
300,0.803783,0.763359,0.001417


#Model III. Support Vector Machine
Suppoer Vector Machine, SVM in short is generally used in classification problems.  
SVM maximizes the space between the decision boundaries and the data so that decrease the chance for mis-classification. 
 * **Strengths**
  * Works well in complicated domain where there is clear margin of separation.
  * We can get nonlinear decision boundary with SVM.
 * **Weakness**
  * The training run is about the cubic of the size of data so SVM trains slowly on large datasets.
  * SVM does not work well with very noisy data where the class are very overlapping. 
  
We have only 395 data samples, which is not a large set so we do not need to concern about the speed problem. Moreover, we have 2 classes to classify and I think SVM is a good model for it.

In [43]:
from sklearn import svm
clf_SVM = svm.SVC()
display(train_different_size(clf_SVM, X_train, y_train, X_test, y_test))

------------------------------------------
Training set size: 100
Training SVC...
Done!
Training time (secs): 0.006
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.877697841727
F1 score for test set: 0.774647887324
------------------------------------------
Training set size: 200
Training SVC...
Done!
Training time (secs): 0.005
Predicting labels using SVC...
Done!
Prediction time (secs): 0.003
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.867924528302
F1 score for test set: 0.781456953642
------------------------------------------
Training set size: 300
Training SVC...
Done!
Training time (secs): 0.030
Predicting labels using SVC...
Done!
Prediction time (secs): 0.006
Predicting labels using SVC...
Done!
Prediction time (secs): 0.002
F1 score for training set: 0.876068376068
F1 score for test set: 0.783783783784


,train_f1_score,test_f1_score,training_time (secs)
training_size,,,
100,0.877698,0.774648,0.006438
200,0.867925,0.781457,0.004841
300,0.876068,0.783784,0.030458


## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

**Answer**  
Let me first wrap up a bit for the experiments I done above.  
The three classifiers are **decision tree, naive bayes and SVM**.  
To decide a model as the best model. I am going to compare the models in terms of accuracy and training time.
  
* **Accuracy**
 * The f1 scores of the three models on 300 data points are 0.62, 0.76 and 0.78 respectively. 
 * 0.76 (NB) and 0.78 (SVM) have only a small difference. We can't judge only by that number, we need to see also the trend of the accuracy as training data size increase.
   * For Naive Bayes, when training data size increase, the test_f1_score decrease from 0.8 to 0.72, then increase to 0.76
   * For SVM, the test_f1_score keep increasing from 0.77 to 0.781, and then to 0.783.
 * So I will say, SVM are more convincing as the best model.
  
  
  
* **Training time**
 * The training time of the three models on 300 data points are 0.001, 0.0007 and 0.006 seconds respectively.
 * NB is the least time-consuming while SVM is the most.
 
 
Although the time spent for SVM is almost 10 times to NB, I would still choose SVM as the best model because accuracy is most important for me and 0.006 seconds is still very quick on my machine. 

After we trained the training set on a SVM, we got a function so that we can make predictions for new data.

**prediction = f(feature1, feature2, feature3, ... , feature48)**

So we can input a vector features to the function, and a prediction number is outputed.  
If the number is positive, it means the student may pass the course and the student does not need early intervention.  
If the number is negative, it means the student may not pass the course so the student may need early intervention.

In [119]:
# TODO: Fine-tune your model and report the best F1 score
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.metrics import make_scorer


finalSelectedModel = DecisionTreeClassifier()

parameters = {'max_depth': [1,2,3,4,5,6,7,8,9,10,11,12],
              'min_samples_split': [1,2,3,4,5,6,7,8,9,10,11,12],
              'min_samples_leaf': [1,2,3,4,5,6,7,8,9,10,11,12]
             }

#cv = StratifiedShuffleSplit(y_train, n_iter=5, test_size=0.2, random_state=20)

def performance_metric(y_true, y_predict):
    """ Calculates and returns the total error between true and predicted values
        based on a performance metric chosen by the student. """
    return f1_score(y_true, y_predict, pos_label='yes')

score_function = make_scorer(performance_metric, greater_is_better=True)


reg = GridSearchCV(estimator=finalSelectedModel, param_grid=parameters, scoring=score_function, cv=3)
reg.fit(X_train, y_train)

bestEstimator = (reg.best_estimator_)
print "______" 
df_final_result = train_predict(bestEstimator, X_train, y_train, X_test, y_test)

print (bestEstimator)
display(df_final_result)

______
------------------------------------------
Training set size: 300
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.001
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.829875518672
F1 score for test set: 0.8
DecisionTreeClassifier(compute_importances=None, criterion='gini',
            max_depth=2, max_features=None, max_leaf_nodes=None,
            min_density=None, min_samples_leaf=6, min_samples_split=1,
            random_state=None, splitter='best')


[300, 0.82987551867219922, 0.80000000000000016, 0.000640869140625]

In [ ]:
[300, 0.83229813664596275, 0.79452054794520555, 0.00650787353515625]

In [115]:
testSVC = svm.SVC(kernel='rbf', C=765, gamma=0.0001)
testSVC2 = svm.SVC(kernel='rbf', C=165, gamma=0.0001)